
# Workshop: Consumir la API de Prometheus con Python + `requests` + `pandas`

**Objetivo:** aprender un patrón limpio para:
1. Autenticarnos y consultar endpoints de Prometheus.  
2. Convertir la respuesta JSON a `DataFrame`.  
3. "Parsear" campos anidados (como `labels`) para análisis/tablas.  

> Todas las celdas incluyen *buenas prácticas* (nomenclatura, manejo de errores, tiempos de espera, etc.).



## 0) Requisitos y consideraciones

- Python 3.10+  
- Librerías: `requests`, `pandas`  
- Credenciales **vía variables de entorno** (evitar hardcodear passwords):  
  - `PROM_USER`
  - `PROM_PASS`
  - `PROM_BASE_URL` (ej.: `http://10.0.0.1:9090`)

Si tu Prometheus usa HTTPS con certificado propio, puedes poner `VERIFY_TLS=False` para desactivar la verificación (solo si entiendes el riesgo).


In [ ]:

# 1) Imports
# -------------------------------------------------------------
from __future__ import annotations

import os
import json
from typing import Any, Dict

import requests
from requests import Session
from requests.auth import HTTPBasicAuth
from requests.exceptions import RequestException, Timeout

import pandas as pd

# Opcional: suprimir warnings SOLO si VERIFY_TLS es False (riesgo de MITM)
import urllib3
VERIFY_TLS = os.getenv("VERIFY_TLS", "true").lower() == "true"
if not VERIFY_TLS:
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [ ]:

# 2) Configuración (usa variables de entorno) 
# -------------------------------------------------------------
PROM_USER = os.getenv("PROM_USER")
PROM_PASS = os.getenv("PROM_PASS")
PROM_BASE_URL = os.getenv("PROM_BASE_URL")  # p.ej. "http://10.88.66.133:9090"

if not PROM_USER or not PROM_PASS or not PROM_BASE_URL:
    raise RuntimeError(
        "Faltan variables de entorno: PROM_USER, PROM_PASS y/o PROM_BASE_URL"
    )

ENDPOINT_TARGETS = f"{PROM_BASE_URL}/api/v1/targets"

DEFAULT_TIMEOUT = (5, 20)  # (connect_timeout, read_timeout) en segundos


In [ ]:

# 3) Helpers HTTP
# -------------------------------------------------------------
def build_session(user: str, password: str, verify_tls: bool = True) -> Session:
    """Construye una requests.Session con autenticación básica y headers estándar."""
    s = requests.Session()
    s.auth = HTTPBasicAuth(user, password)
    s.headers.update({
        "Accept": "application/json",
        "User-Agent": "prom-workshop/1.0"
    })
    s.verify = verify_tls
    return s


def get_json(session: Session, url: str, timeout: tuple[int, int] = DEFAULT_TIMEOUT) -> Dict[str, Any]:
    """Hace GET y devuelve JSON ya validado. Lanza errores claros si algo falla."""
    try:
        resp = session.get(url, timeout=timeout)
        resp.raise_for_status()
        data = resp.json()
    except Timeout as exc:
        raise Timeout(f"Timeout al consultar {url}: {exc}") from exc
    except RequestException as exc:
        raise RuntimeError(f"Error de red al consultar {url}: {exc}") from exc
    except ValueError as exc:
        raise RuntimeError(f"Respuesta no es JSON válido desde {url}: {exc}") from exc

    if not isinstance(data, dict) or "status" not in data:
        raise RuntimeError(f"Estructura inesperada en la respuesta de {url}")
    if data.get("status") != "success":
        raise RuntimeError(f"Prometheus devolvió status != success en {url}: {data}")

    return data


In [ ]:

# 4) Llamada al endpoint /api/v1/targets
# -------------------------------------------------------------
session = build_session(PROM_USER, PROM_PASS, VERIFY_TLS)
targets_json = get_json(session, ENDPOINT_TARGETS)
print("Keys en la raíz:", list(targets_json.keys()))
print("Subclaves en 'data':", list(targets_json.get("data", {}).keys()))


In [ ]:

# 5) JSON -> DataFrame
# -------------------------------------------------------------
active_targets = targets_json.get("data", {}).get("activeTargets", [])
if not isinstance(active_targets, list):
    raise RuntimeError("Formato inesperado: 'activeTargets' no es una lista.")

df_targets = pd.DataFrame(active_targets)
print(f"Filas: {len(df_targets)}, Columnas: {df_targets.shape[1]}")
df_targets.head()



### Notas sobre el DataFrame original
Suele incluir columnas como:
- `discoveredLabels` *(dict)*
- `labels` *(dict)*
- `scrapePool`, `scrapeUrl`, `lastScrape`, etc.

A continuación, **expandimos** `labels` para obtener columnas tabulares.


In [ ]:

# 6) Expandir la columna 'labels' (dict) a columnas
# -------------------------------------------------------------
labels_df = df_targets["labels"].apply(pd.Series)
labels_df = labels_df.add_prefix("label_")

cols_keep = ["scrapePool", "scrapeUrl", "lastScrape", "health", "lastError"]
final_df = pd.concat([df_targets[cols_keep], labels_df], axis=1)

final_df.head(10)



## 7) Ejemplos de filtrado y selección

Al tener las `labels` como columnas, es muy cómodo filtrar/contar.


In [ ]:

if "label_job" in final_df.columns:
    print(final_df["label_job"].value_counts())
    
print("\nTargets con health != 'up':")
print(final_df[final_df["health"] != "up"][["scrapeUrl", "health", "lastError"]].head(10))


In [ ]:

# 8) Guardar resultados
# -------------------------------------------------------------
out_csv = "/mnt/data/prometheus_targets_labels.csv"
out_xlsx = "/mnt/data/prometheus_targets_labels.xlsx"

final_df.to_csv(out_csv, index=False)
final_df.to_excel(out_xlsx, index=False)

out_csv, out_xlsx



## 9) Buenas prácticas aplicadas (y por qué)

- **Variables de entorno** para credenciales y URL → evita exponer passwords en el notebook.
- **`requests.Session`** con **`HTTPBasicAuth`** → reuso de conexiones y autenticación consistente.
- **Timeouts** (`DEFAULT_TIMEOUT`) → el notebook no se queda colgado si el servidor no responde.
- **Validación de respuesta** y manejo de **excepciones** claras.
- **PEP 8**: `snake_case` para variables/funciones, `UPPERCASE` para constantes, indentación de 4 espacios.
- **Comentarios y docstrings** explicando el *por qué*, no solo el *qué*.
- **Separación de responsabilidades**: helpers para red, celdas para análisis.
- **`pd.json_normalize` / `apply(pd.Series)`** para tabular diccionarios.
- **Export a CSV/Excel** para compartir o alimentar otros procesos.



## 10) Ideas para extender en vivo (demo corta)

- Añadir otro endpoint (p.ej. `/api/v1/alerts`) y repetir el patrón.
- Medir tiempos con `%%time` y comparar `apply(pd.Series)` vs. `json_normalize`.
- Construir una función `fetch_targets(base_url) -> DataFrame` y mapear varias sedes.
- Graficar conteos de targets por `job` o por `entorno`.
